In [1]:
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

CAPTION PROCESSING

In [4]:
df = pd.read_csv('/content/drive/MyDrive/flickr dataset/results.csv')

In [5]:
df.head()

,image_name| comment_number| comment,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1000092795.jpg| 0| Two young guys with shaggy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000092795.jpg| 1| Two young,White males are outside near many bushes .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000092795.jpg| 2| Two men in green shirts are...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000092795.jpg| 3| A man in a blue shirt stand...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000092795.jpg| 4| Two friends enjoy time spen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df.apply(lambda row: ''.join(row.dropna().astype(str)), axis=1)

In [7]:
master = {}

In [8]:
for i in df:
    entry = i.split('|')   
    image_name = entry[0]    
    if(len(entry) < 3):
        caption = entry[1] 
    else:
        caption = entry[2] 
    if image_name in master.keys():
        master[image_name].append(caption)
    else:
        master[image_name] = []
        master[image_name].append(caption)

In [9]:
def pre_process(x):
    x = x.lower()
    x = re.sub(r'[^\w\s]', '', x)
    x = re.sub("[^a-z]+", " ", x)
    return x

In [10]:
for i in master:
    captions = master[i]
    master[i] = []
    for x in captions:
        caption = pre_process(x)
        master[i].append(caption)

In [11]:
myKeys = list(master.keys())
myKeys.sort()
sorted_master = {i: master[i] for i in myKeys}

In [12]:
for i in sorted_master:
    captions = sorted_master[i]
    new_captions = []
    for caption in captions:
        new_caption = 'sos ' + caption + ' eos'
        new_captions.append(new_caption)
    sorted_master[i] = new_captions

In [15]:
freq = {}
for i in sorted_master:
    captions = sorted_master[i]
    for caption in captions:
        for word in caption.split():
            freq[word] = freq.get(word, 0) + 1

In [24]:
sorted_freq = sorted(freq.items(), key=lambda x:x[1], reverse = True)
sorted_freq

[('a', 97056),
 ('sos', 57196),
 ('eos', 57196),
 ('in', 29476),
 ('the', 22187),
 ('on', 16103),
 ('and', 15268),
 ('man', 14829),
 ('is', 14548),
 ('of', 13303),
 ('with', 12570),
 ('two', 7573),
 ('woman', 7359),
 ('are', 6844),
 ('to', 5961),
 ('at', 5876),
 ('people', 5678),
 ('wearing', 5464),
 ('an', 5294),
 ('white', 4899),
 ('young', 4865),
 ('dog', 4837),
 ('shirt', 4600),
 ('black', 4446),
 ('boy', 4312),
 ('blue', 3973),
 ('while', 3966),
 ('girl', 3869),
 ('his', 3859),
 ('sitting', 3598),
 ('red', 3463),
 ('playing', 3321),
 ('standing', 3320),
 ('men', 3233),
 ('group', 2737),
 ('front', 2559),
 ('holding', 2473),
 ('water', 2466),
 ('her', 2417),
 ('down', 2401),
 ('child', 2191),
 ('street', 2165),
 ('walking', 2142),
 ('little', 2137),
 ('one', 2079),
 ('three', 2069),
 ('brown', 1958),
 ('by', 1907),
 ('green', 1879),
 ('up', 1844),
 ('children', 1813),
 ('large', 1705),
 ('for', 1698),
 ('through', 1683),
 ('looking', 1660),
 ('yellow', 1645),
 ('women', 1622),
 ('o

In [ ]:
sorted_master

In [ ]:
import pickle
with open("/content/drive/MyDrive/flickr dataset/caption_dict.pkl", "wb") as file:
    pickle.dump(sorted_master, file)

In [ ]:
def count_unique_words(master):
    d = {}
    for i in master:
        captions = master[i]
        for caption in captions:
            for word in caption.split():
                d[word] = d.get(word, 0) + 1
    
    return d

In [ ]:
word_counts = count_unique_words(master)

In [ ]:
updated_word_counts = {}
for i in word_counts:
    if word_counts[i] > 10:
        updated_word_counts[i] = word_counts[i]
        

In [ ]:
all_words = list(updated_word_counts.keys())

LABEL ENCODING

In [ ]:
word_to_index = {}
index_to_word = {}

for i, word in enumerate(all_words):
    word_to_index[word] = i+1
    index_to_word[i+1] = word

In [ ]:
vocab_size = len(word_to_index)

word_to_index['sos'] = vocab_size + 1
index_to_word[vocab_size + 1] = 'sos'

word_to_index['eos'] = vocab_size + 2
index_to_word[vocab_size + 2] = 'eos'

vocab_size = len(word_to_index) + 1

In [ ]:
import pickle
with open("/content/drive/MyDrive/flickr dataset/word_to_index.pkl", "wb") as file:
    pickle.dump(word_to_index, file)
with open("/content/drive/MyDrive/flickr dataset/index_to_word.pkl", "wb") as file:
    pickle.dump(index_to_word, file)

In [ ]:
# Largest preprocess caption
max_length = 0
for i in master:
    captions = master[i]
    for x in captions:
        max_length = max(max_length, len(x))
max_length

400